# PL imaging  paper figures
* Plots that are made in python made here

In [ ]:
# Some imports:
from external_imports import *
from image_process import *
from general_data_process import *
from voltage_dependant import *
from intensity_dependant import *
from copy import deepcopy
from cmcrameri import cm
import scipy.fftpack as sfft
from scipy.ndimage.measurements import label
from scipy.special import lambertw as LambertW
import scipy.constants as sci
from scipy.interpolate import interp1d as inter
import matplotlib
from joblib import Parallel, delayed
import os
from cmcrameri import cm
import mpmath as mp
from copy import deepcopy

# For parallel processing
import multiprocessing
num_cores = multiprocessing.cpu_count()

# Plot parameters
import matplotlib
matplotlib.rcParams['figure.dpi'] = 100
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['font.family'] = "Century Gothic"
from scipy.io import savemat

## Transport layer series
* Plots for all the different transport layers

In [ ]:
datapath = r""
savepath = r""

path_db = path_process(datapath)
keys_to_pop = []
for key in path_db.keys():
    if path_db[key] == []:
        keys_to_pop.append(key)
for key_to_pop in keys_to_pop:
    del path_db[key_to_pop]
    
bandgaps = {}
for key in path_db.keys():
    string = "What's the bandgap of "+key+" in eV?: "
    bandgap = float(input(string))
    bandgaps[key] = bandgap

### Loads data and puts them in dicts:
* Center norm QFLS is given by:
$ Cennter\; norm\; QFLS = \frac{QFLS_{i,j} - <QFLS>}{<QFLS>}$
* This way stuff with different levels of quenching will be comparable 

In [ ]:
plqe_ims = {}
QFLS_ims = {}
QFLS_lists = {}
QFLS_lists_norm = {}
bandgap_per_pix = {}

for key in path_db.keys():
    for pix in path_db[key]:
        filename = find_npy(f"{datapath}/{key}/{pix}/PLQE_oc")[0]
        bandgap_per_pix[f"{key}, {pix}"] = bandgaps[key]
        plqe_ims[f"{key}, {pix}"] = np.load(f"{datapath}/{key}/{pix}/PLQE_oc/{filename}")
        QFLS_ims[f"{key}, {pix}"] =  vocradf(bandgaps[key]) + (sci.k*298/sci.e)*np.log(plqe_ims[f"{key}, {pix}"])     
        
        QFLS_lists[f"{key}, {pix}"] = deepcopy(QFLS_ims[f"{key}, {pix}"]).flatten()
        QFLS_lists_norm[f"{key}, {pix}"] = deepcopy(QFLS_ims[f"{key}, {pix}"]).flatten()
        QFLS_lists_norm[f"{key}, {pix}"] =  (QFLS_lists_norm[f"{key}, {pix}"] - np.mean( QFLS_lists_norm[f"{key}, {pix}"]))/np.mean( QFLS_lists_norm[f"{key}, {pix}"])

### Saves mean QFLS and inter quartile range:

In [ ]:
with open(f"{savepath}\\mean_QFLS.csv",'w',newline='') as file:
    writer = csv.writer(file)
    for key in plqe_ims.keys():
        QFLS = vocradf(bandgap_per_pix[key]) + (sci.k*298/sci.e)*np.log(plqe_ims[key])
        q75, q25 = np.percentile(QFLS.flatten(), [75 ,25])
        iqr = q75 - q25
        
        writer.writerow([key, np.mean(QFLS), iqr, iqr/np.mean(QFLS)])

### Stores flattened count and flattend center norm counts:

In [ ]:
with open(f"{savepath}/histogram.csv",'w',newline='') as file:
    writer = csv.writer(file)
    writer.writerow(QFLS_lists.keys())
    for i in range(len(QFLS_lists[[*QFLS_lists][0]])):
        row = []
        for key in QFLS_lists.keys():
            row.append(QFLS_lists[key][i])
        writer.writerow(row)

with open(f"{savepath}/histogram_norm.csv",'w',newline='') as file:
    writer = csv.writer(file)
    writer.writerow(QFLS_lists_norm.keys())
    for i in range(len(QFLS_lists_norm[[*QFLS_lists_norm][0]])):
        row = []
        for key in QFLS_lists_norm.keys():
            row.append(QFLS_lists_norm[key][i])
        writer.writerow(row)

### Plots center norm images:

In [ ]:
if not os.path.isdir(f"{savepath}/QFLS_centernorm"):
    os.makedirs(f"{savepath}/QFLS_centernorm")

import matplotlib
matplotlib.rcParams['font.size'] = 18
    
lims = {}
for key in QFLS_ims.keys():
    maker = key.split('_')[0]
    std = np.std((QFLS_ims[key]-np.mean(QFLS_ims[key]))/np.mean(QFLS_ims[key]))
    if maker in [*lims]:
        if std > lims[maker]:
            lims[maker] = std
    else:
        lims[maker] = std
            
    
    
for key in QFLS_ims.keys():    
    std = lims[ key.split('_')[0]]
    
    fig, ax = plt.subplots()
    imarr = (QFLS_ims[key]-np.mean(QFLS_ims[key]))/np.mean(QFLS_ims[key])
    im = ax.imshow(imarr, vmin=-0.01, vmax=0.01, cmap=cm.batlow)#, vmin=-4*std, vmax = 4*std, cmap=cm.batlow)
    

    cbar = plt.colorbar(im,pad=0.005,aspect=20)
    cbar.set_label(r"center norm QFLS", rotation=270,labelpad=15)

    scalebar_size = 0.1 # cm
    scalebar_offset = 10 # pix

    scalebar_width_pix = 0.1*pixels_per_cm
    scalebar_height_pix = scalebar_width_pix/10
    scalbar_y_position = imarr.shape[0]-scalebar_offset-scalebar_height_pix

    text_x = scalebar_offset + scalebar_width_pix/2
    text_y = scalbar_y_position

    scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
    ax.add_patch(scalebar)
    txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
    txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

    plt.axis('off')
    plt.savefig(f"{savepath}/QFLS_centernorm/{key}.png", dpi=320, bbox_inches='tight', transparent=True)
    plt.close('all')

* without_axis:

In [ ]:
if not os.path.isdir(f"{savepath}\\QFLS_centernorm_noax"):
    os.makedirs(f"{savepath}\\QFLS_centernorm_noax")

import matplotlib
matplotlib.rcParams['font.size'] = 18
    
lims = {}
for key in QFLS_ims.keys():
    maker = key.split('_')[0]
    std = np.std((QFLS_ims[key]-np.mean(QFLS_ims[key]))/np.mean(QFLS_ims[key]))
    if maker in [*lims]:
        if std > lims[maker]:
            lims[maker] = std
    else:
        lims[maker] = std
            
    
    
for key in QFLS_ims.keys():    
    std = lims[ key.split('_')[0]]
    
    fig, ax = plt.subplots()
    imarr = (QFLS_ims[key]-np.mean(QFLS_ims[key]))/np.mean(QFLS_ims[key])
    im = ax.imshow(imarr, vmin=-0.01, vmax=0.01, cmap=cm.batlow)#, vmin=-4*std, vmax = 4*std, cmap=cm.batlow)
    
    scalebar_size = 0.1 # cm
    scalebar_offset = 10 # pix

    scalebar_width_pix = 0.1*pixels_per_cm
    scalebar_height_pix = scalebar_width_pix/10
    scalbar_y_position = imarr.shape[0]-scalebar_offset-scalebar_height_pix

    text_x = scalebar_offset + scalebar_width_pix/2
    text_y = scalbar_y_position

    scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
    ax.add_patch(scalebar)
    txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
    txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

    plt.axis('off')
    plt.savefig(f"{savepath}\\QFLS_centernorm_noax\\{key}.png", dpi=320, bbox_inches='tight', transparent=True)
    plt.close('all')

### Fourier transformed image (Logged so you can actually see stuff):

In [ ]:
if not os.path.isdir(f"{savepath}\\FT"):
    os.makedirs(f"{savepath}\\FT")
matplotlib.rcParams['font.size'] = 18

for key in QFLS_ims.keys():
    for pix in
    reim = (QFLS_ims[key])
    freq = sfft.fft2(reim)
    freq =  sfft.fftshift(freq) 
      
    fig, ax = plt.subplots()
    imarr = np.log(abs(freq))
    im = ax.imshow(imarr, cmap=cm.batlow, vmin = -6.5, 
                   vmax = 1)
    

    cbar = plt.colorbar(im,pad=0.01,aspect=20)
    cbar.set_label(r"ln(Frequencies)", rotation=270,labelpad=15)

    plt.axis('off')                                                                            
    plt.savefig(f"{savepath}\\FT\\{key}.png", dpi=320, transparent=True, bbox_inches='tight')
    plt.close('all')

## CELLS

### Make 1sun QFLS at oc and coleff
* From intensity dependant data

In [ ]:
cellpath = r"E:\Data\EL_setup\EL_cam_paper\final_set\Plots\new_processed_data"
cell_path_db = path_process(cellpath)


for key in cell_path_db.keys():
    for pix in cell_path_db[key]:
        
        ##### QFLS, oc, 1sun
        oc_files = find_npy(f"{cellpath}\\{key}\\{pix}\\QFLS_int_oc")
        num_suns = [float(i.split('_')[1]) for i in oc_files]
        oc_files = [i for _,i in sorted(zip(num_suns,oc_files))]
        num_suns.sort()
        oc_arrs = [np.load(f"{cellpath}\\{key}\\{pix}\\QFLS_int_oc\\{i}") for i in oc_files]
        
        QFLS_oc_1sun = np.zeros(oc_arrs[0].shape)
        np.save('temp', QFLS_oc_1sun)
        
        def process_row(i):
            arr = np.load('temp.npy', mmap_mode='r+')
            for j in range(oc_arrs[0].shape[1]):
                f = inter(num_suns, [k[i,j] for  k in oc_arrs])
                arr[i,j] = f(1)
        Parallel(n_jobs=num_cores, verbose = 0)(delayed(process_row)(i) for i in range(oc_arrs[0].shape[0]))
        QFLS_oc_1sun = np.load('temp.npy')
        # Delete temp: 
        os.remove('temp.npy')
        np.save(f"{cellpath}\\{key}\\{pix}\\QFLS_oc_1sun",QFLS_oc_1sun)
        
        
        ##### Coleff, 1sun
        oc_files = find_npy(f"{cellpath}\\{key}\\{pix}\\PLQE_oc")
        sc_files = find_npy(f"{cellpath}\\{key}\\{pix}\\PLQE_sc")
        
        num_suns = [float(i.split('_')[1]) for i in oc_files]
        oc_files = [i for _,i in sorted(zip(num_suns,oc_files))]
        num_suns = [float(i.split('_')[1]) for i in sc_files]
        sc_files = [i for _,i in sorted(zip(num_suns,sc_files))]
        num_suns.sort()
        
        oc_arrs = [np.load(f"{cellpath}\\{key}\\{pix}\\PLQE_oc\\{i}") for i in oc_files]
        sc_arrs = [np.load(f"{cellpath}\\{key}\\{pix}\\PLQE_sc\\{i}") for i in sc_files]
        
        col_eff = np.zeros(oc_arrs[0].shape)
        np.save('temp', col_eff)
        
        def process_row(i):
            arr = np.load('temp.npy', mmap_mode='r+')
            for j in range(oc_arrs[0].shape[1]):
                f1 = inter(num_suns, [k[i,j] for  k in oc_arrs])
                f2 = inter(num_suns, [k[i,j] for  k in sc_arrs])
                arr[i,j] = 1 - (f2(1)/f1(1))
        Parallel(n_jobs=num_cores, verbose = 0)(delayed(process_row)(i) for i in range(oc_arrs[0].shape[0]))
        col_eff = np.load('temp.npy')
        # Delete temp: 
        os.remove('temp.npy')
        np.save(f"{cellpath}\\{key}\\{pix}\\col_eff_1sun", col_eff)
        


### Plot cell images

In [ ]:
for key in cell_path_db.keys():
    for pix in cell_path_db[key]:
        basepath = f"{cellpath}\\{key}\\{pix}"
        QFLS_1sun = np.load(f"{basepath}\\QFLS_oc_1sun.npy")
        coleff = np.load(f"{basepath}\\col_eff_1sun.npy")
        sr = np.load(f"{basepath}\\series_resistance.npy")
        nid = np.load(f"{basepath}\\ideality.npy")
        
        
        ###############################
        #QFLS
        fig, ax = plt.subplots()

        im = ax.imshow(QFLS_1sun, cmap=cm.batlow, vmin=1.11, vmax=1.145)
        cbar = plt.colorbar(im,pad=0.01,aspect=20)
        cbar.set_label('QFLS (eV)', rotation=270,labelpad=25)

        scalebar_size = 0.1 # cm
        scalebar_offset = 10 # pix

        scalebar_width_pix = 0.1*pixels_per_cm
        scalebar_height_pix = scalebar_width_pix/10
        scalbar_y_position = QFLS_1sun.shape[0]-scalebar_offset-scalebar_height_pix

        text_x = scalebar_offset + scalebar_width_pix/2
        text_y = scalbar_y_position

        scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
        ax.add_patch(scalebar)
        txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
        txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

        plt.axis('off')

        plt.savefig(f"{cellpath}\\{key}\\{pix}\\QFLS_oc_1sun.png", transparent = True, bbox_inches='tight', dpi=300)
        print(key, "QFLS", np.mean(QFLS_1sun))
        plt.close('all')
        
        
        #################################
        #Coleff
        
        fig, ax = plt.subplots()

        im = ax.imshow(coleff*100, cmap=cm.batlow, vmin=60, vmax=100)
        cbar = plt.colorbar(im,pad=0.01,aspect=20)
        cbar.set_label('$\eta_{col}$(%)', rotation=270,labelpad=25)

        scalebar_size = 0.1 # cm
        scalebar_offset = 10 # pix

        scalebar_width_pix = 0.1*pixels_per_cm
        scalebar_height_pix = scalebar_width_pix/10
        scalbar_y_position = QFLS_1sun.shape[0]-scalebar_offset-scalebar_height_pix

        text_x = scalebar_offset + scalebar_width_pix/2
        text_y = scalbar_y_position

        scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
        ax.add_patch(scalebar)
        txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
        txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

        plt.axis('off')

        plt.savefig(f"{cellpath}\\{key}\\{pix}\\col_eff_1sun.png", transparent = True, bbox_inches='tight', dpi=300)
        print(key, "coleff", np.mean(coleff*100))
        plt.close('all')
        
        
        #################################
        #ideality factor
        
        fig, ax = plt.subplots()

        im = ax.imshow(nid, cmap=cm.batlow, vmin =1.1, vmax=1.7)
        cbar = plt.colorbar(im,pad=0.01,aspect=20)
        cbar.set_label('$n_{id}$', rotation=270,labelpad=25)

        scalebar_size = 0.1 # cm
        scalebar_offset = 10 # pix

        scalebar_width_pix = 0.1*pixels_per_cm
        scalebar_height_pix = scalebar_width_pix/10
        scalbar_y_position = QFLS_1sun.shape[0]-scalebar_offset-scalebar_height_pix

        text_x = scalebar_offset + scalebar_width_pix/2
        text_y = scalbar_y_position

        scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
        ax.add_patch(scalebar)
        txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
        txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

        plt.axis('off')

        plt.savefig(f"{cellpath}\\{key}\\{pix}\\ideality.png", transparent = True, bbox_inches='tight', dpi=300)
        print(key, "id", np.mean(nid))
        plt.close('all')

# TOC

In [ ]:
path = r"E:\Data\EL_setup\EL_cam_paper\final_set_data\processed_data\Yen_and_aug_cells_processed\004A08\6\col_eff_1sun.npy"
arr = np.load(path)
from scipy.ndimage import gaussian_filter

inc = 0.02

arr = gaussian_filter(arr, sigma=3)


for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        if arr[i,j]<=0.85:
            arr[i,j] = 0.85
        
        if arr[i,j]>=0.91:
            arr[i,j] = 0.91
        
        arr[i,j] = round(arr[i,j] / inc) * inc
        

x = np.arange(arr.shape[1])
y = np.arange(arr.shape[0])
Y, X = np.meshgrid(y, x)


plt.gca().set_aspect("equal")
#plt.imshow(gaussian_filter(arr, sigma=1.6), vmin=0.85, vmax=0.91, cmap=cm.buda)
c1 = plt.contourf(X, Y, arr.transpose(), 30, cmap=cm.buda, antialiased=False)
c2 = plt.contour(X, Y, arr.transpose(), 30, colors=['k'],linewidths=[0.2])
#plt.axis('equal')
cbar = plt.colorbar(c1, pad=0.01)
cbar.set_ticks([])
plt.axis('off')

plt.savefig(r"E:\Data\EL_setup\EL_cam_paper\final_set_data\Plots\TOC\colevv.png", bbox_inches='tight', transparent=True, dpi=300)


# Series/shunt impact calculations

In [ ]:
matplotlib.rcParams['font.size'] = 20
matplotlib.rcParams['font.family'] = "Century Gothic"

Iph, I0 = (0.01856709832455347,1e-14) # reasonable values
Vth = sci.k * 298 / sci.e
n = 1.4

def I(V, Rsh, Rs, Iph=Iph):
    icalc = (
        Rs * (I0 * Rsh + Iph * Rsh - V)
        - Vth
        * n
        * (Rs + Rsh)
        * LambertW(
            Rs
            * np.exp(
                (
                    Rs * V
                    + Rs * (I0 * Rsh + Iph * Rsh - V)
                    + Rsh * V
                    + (Vth * n * (Rs + Rsh)) * np.log(I0 * Rsh)
                )
                / (Vth * n * (Rs + Rsh))
            )
            / (Vth * n * (Rs + Rsh))
        )
    ) / (Rs * (Rs + Rsh))

    return np.real(icalc)

def Voc(Rsh, Rs):
    Rsh*(I0 + Iph) - Vth*n*LambertW(float(I0*Rsh*mp.exp(Rsh*(I0 + Iph)/(Vth*n))/(Vth*n)))

Calculate arrays and save them:

In [ ]:
rs_range = np.geomspace(0.01, 1000,1000)
rsh_range = np.geomspace(1, 1e4, 10000)
eg_arr = np.zeros((len(rs_range), len(rsh_range)))

np.save('Vrsh_oc', eg_arr)
np.save('Vrs_sc', eg_arr)
np.save('Vrsh_sc', eg_arr)

np.save('Irsh_oc', eg_arr)
np.save('Id_oc', eg_arr)

np.save('Irs_sc', eg_arr)
np.save('Irsh_sc', eg_arr)
np.save('Id_sc', eg_arr)

def process_row(i):
    Vrsh_oc_arr = np.load('Vrsh_oc.npy', mmap_mode='r+')
    Vrs_sc_arr = np.load('Vrs_sc.npy', mmap_mode='r+')
    Vrsh_sc_arr = np.load('Vrsh_sc.npy', mmap_mode='r+')

    Irsh_oc_arr = np.load('Irsh_oc.npy', mmap_mode='r+')
    Id_oc_arr = np.load('Id_oc.npy', mmap_mode='r+')

    Irs_sc_arr = np.load('Irs_sc.npy', mmap_mode='r+')
    Irsh_sc_arr = np.load('Irsh_sc.npy', mmap_mode='r+')
    Id_sc_arr = np.load('Id_sc.npy', mmap_mode='r+')
    
    Rs = rs_range[i] 
    for j, Rsh in enumerate(rsh_range):
        Vrange = np.arange(0, 1.3,0.01)
        Irange = I(Vrange, Rsh, Rs)
        voc = inter(Irange, Vrange, bounds_error=False, fill_value=np.finfo(float).eps)(0)
        #print(voc)

        ###oc
        Il_oc = 0
        Vl_oc = voc
        Vrs_oc = 0
        Vrsh_oc = Vl_oc + Vrs_oc
        
        if Vrsh_oc >= np.finfo(float).eps:
            Vrsh_oc_arr[i,j] = Vrsh_oc
        else:
            Vrsh_oc_arr[i,j] = np.finfo(float).eps
        
        Ish_oc = Vrsh_oc / Rsh
        
        if Ish_oc > np.finfo(float).eps:
            Irsh_oc_arr[i,j] = Ish_oc
        else:
            Irsh_oc_arr[i,j] = np.finfo(float).eps

        Id_oc = Iph - (Ish_oc + Il_oc)
        
        if Id_oc > np.finfo(float).eps:
            Id_oc_arr[i,j] = Id_oc 
        else:
            Id_oc_arr[i,j] = np.finfo(float).eps

        ###sc
        Il_sc = I(0, Rsh, Rs)
        
        if Il_sc > np.finfo(float).eps:
            Irs_sc_arr[i,j] = Il_sc
        else:
            Irs_sc_arr[i,j] = np.finfo(float).eps
        Vl_sc = 0

        Vrs_sc = Il_sc * Rs
        
        if Vrs_sc > np.finfo(float).eps:
            Vrs_sc_arr[i,j] = Vrs_sc
        else:
            Vrs_sc_arr[i,j] = np.finfo(float).eps
        
        Vrsh_sc = Vl_sc + Vrs_sc 
        
        if Vrsh_sc >= np.finfo(float).eps:
            Vrsh_sc_arr[i,j] = Vrsh_sc
        else:
            Vrsh_sc_arr[i,j] = np.finfo(float).eps
        Ish_sc = Vrsh_sc / Rsh
        
        if Ish_sc > np.finfo(float).eps:
            Irsh_sc_arr[i,j] = Ish_sc
        else:
            Irsh_sc_arr[i,j] = np.finfo(float).eps
            
            
        Id_sc = Iph - (Ish_sc + Il_sc)
        
        if Id_sc > np.finfo(float).eps:
            Id_sc_arr[i,j] = Id_sc
        else:
            Id_sc_arr[i,j] = np.finfo(float).eps

Parallel(n_jobs=8, verbose = 0)(delayed(process_row)(i) for i in range(len(rs_range)))

Load:

In [ ]:
rs_range = np.geomspace(0.01, 1000,1000)
rsh_range = np.geomspace(1, 1e4, 10000)
eg_arr = np.zeros((len(rs_range), len(rsh_range)))

Vrsh_oc_arr = np.load('Vrsh_oc.npy')
Vrsh_oc_arr= np.flip(Vrsh_oc_arr, 0)
Vrs_sc_arr = np.load('Vrs_sc.npy')
Vrs_sc_arr= np.flip(Vrs_sc_arr, 0)
Vrsh_sc_arr = np.load('Vrsh_sc.npy')
Vrsh_sc_arr= np.flip(Vrsh_sc_arr, 0)
Irsh_oc_arr = np.load('Irsh_oc.npy')
Irsh_oc_arr= np.flip(Irsh_oc_arr, 0)
Id_oc_arr = np.load('Id_oc.npy')
Id_oc_arr= np.flip(Id_oc_arr, 0)
Irs_sc_arr = np.load('Irs_sc.npy')
Irs_sc_arr= np.flip(Irs_sc_arr, 0)
Irsh_sc_arr = np.load('Irsh_sc.npy')
Irsh_sc_arr= np.flip(Irsh_sc_arr, 0)
Id_sc_arr = np.load('Id_sc.npy')
Id_sc_arr= np.flip(Id_sc_arr, 0)

matplotlib.rcParams.update({'font.size': 18})

True Collection efficiency

In [ ]:
plt.imshow(1-(Id_sc_arr+Irsh_sc_arr)/(Id_oc_arr+Irsh_oc_arr), aspect = Id_sc_arr.shape[1]/Id_sc_arr.shape[0], cmap=cm.batlow, vmin=0,vmax=1)

x_list = [np.argmin(abs(rsh_range-i)) for i in [1,100, 10000]]
x_label_list = np.around(rsh_range[x_list])
plt.xticks(x_list,x_label_list)

def round_to_1(x):
    return round(x, -int(np.floor(np.log10(abs(x)))))

y_list = [np.argmin(abs(rs_range-i)) for i in [0.01,1, 100]]
y_label_list = [round_to_1(i) for i in rs_range[y_list]]
y_list = [len(rs_range)-np.argmin(abs(rs_range-i)) for i in [0.01,1, 100]]
plt.yticks(y_list,y_label_list)

plt.xlabel("Rsh ($\Omega$cm$^2$)")
plt.ylabel("Rs ($\Omega$cm$^2$)")

cbar = plt.colorbar()
cbar.ax.set_ylabel("1 - (J$_{D+sh}$)|$_{sc}$ /(J$_{D+sh}$)|$_{oc}$")
# plt.tight_layout()
plt.savefig("True_coleff_d_plud_sh.png",dpi=300, bbox_inches='tight')
plt.show()


plt.imshow(Irs_sc_arr/Iph, aspect = Id_sc_arr.shape[1]/Id_sc_arr.shape[0], cmap=cm.batlow, vmin=0,vmax=1)

x_list = [np.argmin(abs(rsh_range-i)) for i in [1,100, 10000]]
x_label_list = np.around(rsh_range[x_list])
plt.xticks(x_list,x_label_list)

def round_to_1(x):
    return round(x, -int(np.floor(np.log10(abs(x)))))

y_list = [np.argmin(abs(rs_range-i)) for i in [0.01,1, 100]]
y_label_list = [round_to_1(i) for i in rs_range[y_list]]
y_list = [len(rs_range)-np.argmin(abs(rs_range-i)) for i in [0.01,1, 100]]
plt.yticks(y_list,y_label_list)

plt.xlabel("Rsh ($\Omega$cm$^2$)")
plt.ylabel("Rs ($\Omega$cm$^2$)")

cbar = plt.colorbar()
cbar.ax.set_ylabel("J$_{sc}$/J$_G$")
plt.savefig("True_coleff_Jsc_by_JG.png",dpi=300, bbox_inches='tight')
plt.show()

Collection Quality

In [ ]:
plt.imshow(1-Id_sc_arr/Id_oc_arr, aspect = Id_sc_arr.shape[1]/Id_sc_arr.shape[0], cmap=cm.batlow, vmin=0,vmax=1)


x_list = [np.argmin(abs(rsh_range-i)) for i in [1,100, 10000]]
x_label_list = np.around(rsh_range[x_list])
plt.xticks(x_list,x_label_list)

def round_to_1(x):
    return round(x, -int(np.floor(np.log10(abs(x)))))

y_list = [np.argmin(abs(rs_range-i)) for i in [0.01,1, 100]]
y_label_list = [round_to_1(i) for i in rs_range[y_list]]
y_list = [len(rs_range)-np.argmin(abs(rs_range-i)) for i in [0.01,1, 100]]
plt.yticks(y_list,y_label_list)

plt.xlabel("Rsh ($\Omega$cm$^2$)")
plt.ylabel("Rs ($\Omega$cm$^2$)")

cbar = plt.colorbar()
cbar.ax.set_ylabel("Q$_{col}$")
plt.savefig("meas_Q_col.png",dpi=300, bbox_inches='tight')
plt.show()

error:

In [ ]:
diff = (1-Id_sc_arr/Id_oc_arr) - (Irs_sc_arr/Iph)
scale = (1-Id_sc_arr/Id_oc_arr)
error = diff/scale

plt.imshow(error*100, aspect = Id_sc_arr.shape[1]/Id_sc_arr.shape[0], cmap=cm.batlow, vmin=0,vmax=100)


x_list = [np.argmin(abs(rsh_range-i)) for i in [1,100, 10000]]
x_label_list = np.around(rsh_range[x_list])
plt.xticks(x_list,x_label_list)

def round_to_1(x):
    return round(x, -int(np.floor(np.log10(abs(x)))))

y_list = [np.argmin(abs(rs_range-i)) for i in [0.01,1, 100]]
y_label_list = [round_to_1(i) for i in rs_range[y_list]]
y_list = [len(rs_range)-np.argmin(abs(rs_range-i)) for i in [0.01,1, 100]]
plt.yticks(y_list,y_label_list)

plt.xlabel("Rsh ($\Omega$cm$^2$)")
plt.ylabel("Rs ($\Omega$cm$^2$)")

cbar = plt.colorbar()
cbar.ax.set_ylabel("Q$_{col}$ error (%)")

for i in range(int(error.shape[0]/4)):
    for j in range(int(error.shape[1]/4)):
        error[i,j] = np.nan
        

plt.contour(abs(error),colors='w', levels=[0,0.1], linewidth=6)
plt.contour(abs(error),colors='y', levels=[0,0.2], linewidth=6)

#####
# detection limit 
detection_limit = -1*np.ones(Id_oc_arr.shape)

for i in range(Id_oc_arr.shape[0]):
    for j in range(Id_oc_arr.shape[1]):
        if Id_oc_zeroed[i,j]/Iph <= 1e-3:
            detection_limit[i,j] = 0
            
plt.contour(detection_limit,colors='aqua', levels=[-1,0], linewidth=6)

plt.savefig("Q_col_err_percentage.png",dpi=300, bbox_inches='tight')
plt.show()


# matplotlib.rcParams['font.size'] = 20
# matplotlib.rcParams['font.family'] = "Century Gothic"